In [7]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import re
from joblib import dump, load

PREFIX='a video of '
events = [
    "Riding",
    "Fighting",
    "Playing",
    "Running",
    "Lying",
    "Chasing",
    "Jumping",
    "Falling",
    "Guiding",
    "Stealing",
    "Littering",
    "Tripping",
    "Pickpockering",
]
description = [
    "a person riding a bicycle on the street",  # Added context
    "multiple people engaged in a physical fight",  # More specific than "fighting"
    "a group of people playing a sport together",  # Added "sport" for visual clarity
    "a person running",  # Added context
    "a person lying motionless on the ground",  # "Motionless" helps distinguish from falling
    "a person aggressively chasing another person",  # "Aggressively" adds distinction
    "a person jumping high in the air with both feet",  # More specific than just "jumping"
    "a person accidentally falling to the ground",  # "Accidentally" helps distinguish
    "a person gently guiding another person by the arm",  # Added detail
    "a person stealing other person",  # More specific than "stealing"
    "a person deliberately throwing garbage on the ground",  # "Deliberately" adds clarity
    "a person tripping over an obstacle",  # More descriptive
    "a person pickpocketing a wallet from someone's pocket",  # Very specific
]
normal_prompt = "a normal view (persons walking or standing)"
description.append(normal_prompt)
description=[PREFIX +desc for desc in description]
events_description = {event: desc for event, desc in zip(events, description)}

In [8]:
'''    thresholds = {
    "a person riding a bicycle on the street":0.01,
    "multiple people engaged in a physical fight":0.1,
    "a group of people playing a sport together":0.01,
    "a person running":0.01,
    "a person lying motionless on the ground":0.65,
    "a person aggressively chasing another person":0.48,
    "a person jumping high in the air with both feet":0.1,
    "a person accidentally falling to the ground":0.01,
    "a person gently guiding another person by the arm":0.01,
    "a person stealing other person":0.35,
    "a person deliberately throwing garbage on the ground":0.01,
    "a person tripping over an obstacle":0.86,
    "a person pickpocketing a wallet from someone's pocket":0.2,
    'a normal view (persons walking or standing)':0.0
    } 
    scores = {event: prob for event, prob in zip(predicted_event, probability) if prob > 0}'''
def check_precision(frames_number, name, predicted_events, event, description, prompts, mode, probabilities):
    # TN Prediction and Reality are true
    # TN Prediction and Reality are false
    # FN Prediction is false and Reality is true
    # FP Prediction is true and Reality is false
    normal_class = PREFIX + "a normal view (persons walking or standing)"
    #
    all_classes = description
    frames = np.load("/home/ubuntu/Database/ALL/GT/gt_ALL.npz")
    frames= frames[name]
    frames = np.append(frames, frames[-1])
    # Create a dictionary to convert class names to numeric indices
    class_to_idx = {cls: idx for idx, cls in enumerate(all_classes)}
    # Example: {"a normal view...": 0, "a person riding...": 1, ...}
    num_classes = len(all_classes)  # 14
    cm = np.zeros((num_classes, num_classes), dtype=int)
    prompts = [prompt.lower().split('.')[0] for prompt in prompts]
    for i in range(len(predicted_events)):
        # Get ground truth
        is_anomaly = frames[frames_number[i] - 1]  # 0 or 1
        # Determine true class using EVENT when anomaly exists
        true_class = event if is_anomaly == 1 else normal_class
        thresholds = {
                        "a person riding a bicycle on the street": 0.01,
                        "multiple people engaged in a physical fight": 0.1,
                        "a group of people playing a sport together": 0.01,
                        "a person running": 0.25,
                        "a person lying motionless on the ground": 0.65,
                        "a person aggressively chasing another person": 0.62,
                        "a person jumping high in the air with both feet": 0.3,
                        "a person accidentally falling to the ground": 0.01,
                        "a person gently guiding another person by the arm": 0.38,
                        "a person stealing other person": 0.5,
                        "a person deliberately throwing garbage on the ground": 0.2,
                        "a person tripping over an obstacle": 0.85,
                        "a person pickpocketing a wallet from someone's pocket": 0.1,
                        "a normal view (persons walking or standing)": 0.0,
                    }
        scores = {event: prob for event, prob in zip(predicted_events[i], probabilities[i]) if prob > 0}
        scores = {
                        event: prob
                        for event, prob in zip(predicted_events[i], probabilities[i]) 
                    if prob > thresholds[event.split(PREFIX)[-1]]
                    }
        try:
            top3 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:3]
            top3_events = [event for event, _ in top3]
            top5 = sorted(scores.items(), key=lambda x: x[1], reverse=True)[:5]
            top5_events = [event for event, _ in top5]
            if (true_class not in top3_events) and (is_anomaly == 1) and (true_class in top5_events):
                print(f"True class: {true_class}, Name {name}, Frame {frames_number[i]}\n {[probabilities[i]]}")
                for evt, scr in scores.items():
                    print(f"  Predicted: {evt}, Score: {scr:.4f}")
                print(f"  Top 3 predicted events: {top3_events}\n  Top 5 predicted events: {top5_events}\n\n\n\n")
        except:
            pass
    '''scores_dict = {
                        event: prob
                        for event, prob in zip(predicted_events[i], probabilities[i]) 
                    if prob > thresholds[event.split(PREFIX)[-1]]
                    }
        scores = {event: prob for event, prob in zip(predicted_events[i], probabilities[i]) if prob > 0}
        try:
            max_event = max(scores, key=scores.get)
            max_event_thres = max(scores_dict, key=scores_dict.get)
            if is_anomaly==1 and true_class in scores.keys() and max_event!=true_class and max_event_thres==true_class:
                print(f"True class: {true_class}, Name {name}, Frame {frames_number[i]}\n {[probabilities[i]]}")
                for evt, scr in scores_dict.items():
                    print(f"  Predicted: {evt}, Score: {scr:.4f}")
                print(f"  Max predicted event: {max_event_thres} with score {scores[max_event_thres]:.4f}\n\n\n\n")
        except:
            pass'''

rute_stored="/home/ubuntu/Tesis/Storage/ScoreTOP5Less"
files=os.listdir(rute_stored)
files = [f for f in files if f.endswith('.npy')]
for file in files:
    name= file.split('_CLIP_')[0]
    mode=int(file.split('_CLIP_')[1][0])
    event=file.split('_CLIP_')[1][2:].split('.')[0]
    info_stored=np.load(f"{rute_stored}/{file}", allow_pickle=True)
    frames_number=info_stored[0]
    predicted_events=info_stored[1]
    prompts=info_stored[2]
    probabilities=info_stored[3]
    check_precision(frames_number, name, predicted_events, event,description, prompts, mode,probabilities)


True class: a video of a person running, Name 4_110_1, Frame 265
 [[0, 0, 0, 0.5125945731152112, 0.2835358309986958, 0.5669202392583319, 0, 0, 0.5719605433893306, 0.6695354185537091, 0.4324710799732038, 0.8248462677001953, 0.6803713653389423, 0]]
  Predicted: a video of a person running, Score: 0.5126
  Predicted: a video of a person gently guiding another person by the arm, Score: 0.5720
  Predicted: a video of a person stealing other person, Score: 0.6695
  Predicted: a video of a person deliberately throwing garbage on the ground, Score: 0.4325
  Predicted: a video of a person pickpocketing a wallet from someone's pocket, Score: 0.6804
  Top 3 predicted events: ["a video of a person pickpocketing a wallet from someone's pocket", 'a video of a person stealing other person', 'a video of a person gently guiding another person by the arm']
  Top 5 predicted events: ["a video of a person pickpocketing a wallet from someone's pocket", 'a video of a person stealing other person', 'a video 